## load library

In [689]:
import os
import pathlib
import random
import shutil
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import scipy.stats
from sklearn import metrics, preprocessing
import pandas as pd
from keras.utils import audio_dataset_from_directory
from scipy.io import wavfile
from IPython import display
import joblib
import soundfile as sf


from imblearn.over_sampling import RandomOverSampler

# # Define the path to the dataset directory
# DATASET_PATH = "C:/Users/ITPKL/Desktop/pydev/ccw_sound"
# data_dir = pathlib.Path(DATASET_PATH)

# # Create training and validation dataset
# train_ds, val_ds =  audio_dataset_from_directory(
#     directory=data_dir,
#     batch_size=64,
#     validation_split=0.3,
#     seed=0,
#     output_sequence_length=16000,
#     subset='both'
# )

# # Retrieve the class names from the training dataset
# label_names = np.array(train_ds.class_names)
# label_names_val = np.array(val_ds.class_names)


# # Print the label names
# print()
# print("label names training:", label_names)
# print("label names testing:", label_names_val)

## input file pipeline (test)/ membaca satu file audio dari fungsi utama dan mengembalikan dalam bentuk numpy

In [690]:
# Define a function to remove singleton dimensions from audio tensor
# def squeeze(audio, labels):
#     audio = tf.squeeze(audio, axis=-1)
#     return audio, labels

# # Apply the squeeze function to the training and validation datasets
# train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
# val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

# Define a function to read WAV files and convert to FFT
def read_wav_file(file_path, fft_length=1024):
    try:
        sample_rate, audio = wavfile.read(file_path)
        if len(audio) == 0:
            return None
        return np.fft.fft(audio)[:fft_length]
    except Exception as e:
        print(f"Error reading WAV file: {file_path}")
        print(f"Error message: {str(e)}")
        return None

## convert fft data to dataframe

In [691]:
def dframe(fmd, pm):
    # Create a DataFrame with the magnitude of the FFT outputs and numerical labels
    fft_magnitude_df = pd.DataFrame(fmd)
    mask_df = pd.DataFrame(pm)
    # Rename the columns of the DataFrame to fft1, fft2, ...
    # new_columns = [f'fft{n}' for n in range(1, fft_length + 1)]
    # fft_magnitude_df.columns = new_columns

    # Add a new column 'Label' to the DataFrame based on the folder name
    # fft_magnitude_df['label'] = labels

    # Display the DataFrame
    # display.display(fft_magnitude_df)

    # fft_magnitude_tuples = [tuple(row) for row in fft_magnitude_df.values]
    # random.shuffle(fft_magnitude_tuples)
    # Display the shuffled list of tuples
        # for tup in fft_magnitude_tuples:
        # print(tup)
    
    
    
    return fft_magnitude_df, mask_df

    

## proses 1 file audio yang sudah di preprocess menjadi fft

In [692]:

# Process a single WAV file
def proses_file_audio(file_path, fft_length=1024):
    fft_magnitude_data = []
    masks = []
    
    # masuk fungsi baca file untuk mengembalikan dalam bentuk data fft 
    fft = read_wav_file(file_path, fft_length)
    if fft is not None:
        fft_magnitude = np.abs(fft)
        mask = np.ones_like(fft_magnitude)
        mask[len(fft_magnitude):] = 0
        fft_magnitude_padded = np.pad(fft_magnitude, (0, fft_length - len(fft_magnitude)))
        mask_padded = np.pad(mask, (0, fft_length - len(mask)))
        
        # Append the padded FFT magnitude and mask to their respective lists
        fft_magnitude_data.append(fft_magnitude_padded)
        masks.append(mask_padded)

    # Initialize an empty list to store the padded masks
    padded_masks = [np.pad(mask, (0, fft_length - len(mask))) for mask in masks]

    # Find the maximum length among all padded masks
    max_length = max(len(mask) for mask in padded_masks)

    # Pad each mask to the maximum length
    padded_masks = [np.pad(mask, (0, max_length - len(mask))) for mask in padded_masks]

    # Convert the lists to numpy arrays
    fft_magnitude_data = np.array(fft_magnitude_data)
    padded_masks = np.array(padded_masks)

    # Print the shapes of the resulting arrays
    print("FFT Magnitude Data Shape:", fft_magnitude_data.shape)
    print("Masks Shape:", padded_masks.shape)

    
    hasil_magnitude , hasil_padded = dframe(fft_magnitude_data, padded_masks)
    


    # belum terbentuk numpy array dan masih dalam bentuk dataframe
    print(hasil_magnitude)
    print()
    print(hasil_padded) #padded mask dari fungsi
    print('===================================================================')

    #sudah terbentuk numpy array  
    print(fft_magnitude_data)
    print()
    print(padded_masks)
    print()

    # import file csv
    X_coba = pd.read_csv('X_coba.csv')
    y_coba = pd.read_csv('y_coba.csv') # ubah disini
    X_train_resampled = pd.read_csv('X_train_resampled.csv')


    from sklearn.preprocessing import StandardScaler
    sc_X = StandardScaler()

    X_train_resampled = sc_X.fit_transform(X_train_resampled)
    X_coba = sc_X.transform(X_coba)
    scaled_input = sc_X.transform(hasil_magnitude)

    # Load the model
    model = joblib.load('C:/Users/ITPKL/Desktop/pydev/model20240130.pkl')

    # Make predictions
    y_pred = model.predict(scaled_input) # ganti disini untuk melihat test
    # print(y_pred) 

    
    if y_pred == 0:
        print("Prediksi model: NG")
    else:
        print("Prediksi model: OK")

    # from sklearn.metrics import accuracy_score

    # accuracy = accuracy_score(y_coba, y_pred)

    # print("Accuracy train:", accuracy)

    # from sklearn.metrics import classification_report, confusion_matrix

    # print(classification_report(y_coba,y_pred,zero_division=1))
    # cm = confusion_matrix(y_coba, y_pred)

    # sns.heatmap(cm, annot=True, yticklabels=True,cbar=True)


In [693]:
# manggil input
proses_file_audio('testing_model/test_ok/CCW_346.wav')

FFT Magnitude Data Shape: (1, 1024)
Masks Shape: (1, 1024)
     0             1             2            3            4          5     \
0  8032.0  11460.938428  11421.040877  7779.445288  7531.039369  6576.5519   

          6            7            8            9     ...        1014  \
0  4727.083644  3630.904414  1505.560959  2601.330374  ...  281.149782   

         1015        1016        1017       1018        1019        1020  \
0  250.759502  173.559206  206.235498  293.06516  360.332295  736.719573   

         1021      1022        1023  
0  220.779597  373.8634  255.100736  

[1 rows x 1024 columns]

   0     1     2     3     4     5     6     7     8     9     ...  1014  \
0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   1.0   

   1015  1016  1017  1018  1019  1020  1021  1022  1023  
0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  

[1 rows x 1024 columns]
[[ 8032.         11460.9384282  11421.04087667 ...   220.7795968
    373.86340037   25

c:\Users\ITPKL\Desktop\pydev\venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [694]:
# Function to read audio, transform to FFT, and process with the ML model
# def analyze_audio():
    
    # audio_data, sample_rate = sf.read('recorded_audio.wav')

    # # Apply noise reduction
    # # reduced_noise = nr.reduce_noise(audio_clip=audio_data, noise_clip=noise_data)
    
    # if sample_rate != 44100:
    #     audio_data = sf.resample(audio_data, 44100)
    
    # # Mengubah audio menjadi mono jika perlu
    # if len(audio_data.shape) > 1:
    #     audio_data = np.mean(audio_data, axis=1)
    
    # # Normalisasi data audio
    # audio_data = audio_data / np.max(np.abs(audio_data))
    # # ======================================================
    # # audio_data = np.frombuffer(stream.read(RATE*5), dtype=np.int16)  # Record for 1 second
    # # fft_data = np.abs(np.fft.fft(audio_data))[:RATE // 2]
    # # ===========================================
    # # Memisahkan audio menjadi chunk-chunk kecil
    # chunk_size = 2048  # Increase chunk_size to match the expected number of features
    # num_chunks = len(audio_data) // chunk_size
    # chunks = np.array_split(audio_data[:num_chunks * chunk_size], num_chunks)
    
    # # Memproses setiap chunk dan membuat prediksi
    # for chunk in chunks:
    #     fft_data = np.abs(np.fft.fft(chunk))[:chunk_size // 2]
        
    #     # Mengubah format data FFT sesuai dengan model yang dilatih
    #     input_data = fft_data.reshape(1, -1)  # Karena model membutuhkan input 2D
        
    #     # Menggunakan model untuk membuat prediksi
    #     prediction = model.predict(input_data)

In [695]:

# def dframe ():
#     # Create a DataFrame with the magnitude of the FFT outputs and numerical labels
#     fft_magnitude_df = pd.DataFrame(fft_magnitude_data)

#     # Rename the columns of the DataFrame to fft1, fft2, ...
#     # new_columns = [f'fft{n}' for n in range(1, fft_length + 1)]
#     # fft_magnitude_df.columns = new_columns

#     # Add a new column 'Label' to the DataFrame based on the folder name
#     # fft_magnitude_df['label'] = labels

#     # Display the DataFrame
#     # display.display(fft_magnitude_df)

In [696]:
# # Takes all fft values as X and drops the Label
# X = fft_magnitude_df.drop(['label'], axis=1)
# # y = fft_magnitude_df['label']

# input_1_image = np.array(X)
# label_1_image = np.array(y)


In [697]:

# Splits the datasset into 80% training set and 20% test set
# from sklearn.model_selection import train_test_split
# Splits the dataset into 80% training set and 20% test set
# input_latih, input_coba, label_latih, label_coba = train_test_split(input_1_image, label_1_image, test_size=0.2, random_state=0)

# Perform oversampling using RandomOverSampler
# oversampler = RandomOverSampler()
# X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)




In [698]:
# Perform feature scaling using StandardScaler
# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# input_model = sc_X.fit_transform(input_coba)


# Train the SVM (Support Vector Machine) classifier
# from sklearn.svm import SVC
# from sklearn.metrics import classification_report, confusion_matrix


# load model
# classifier = SVC(kernel='poly',gamma="scale", C=2.0)
# classifier.fit(X_train_resampled, y_train_resampled)